# Importação das bibliotecas

In [58]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TRT05\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importação dos dados dos livros

In [59]:
df = pd.read_csv("livros.csv")
df['descricao'] = df['descricao'].astype(str)
df[['titulo', 'autor', 'descricao']]

,titulo,autor,descricao
0,Orçamento sem falhas,Nath Finanças,"Quando o assunto é dinheiro, não dá para achar..."
1,Minha Sombria Vanessa,Kate Elizabeth Russell,Elogiado por Gillian Flynn e considerado um do...
2,Recursão,Blake Crouch,E se um dia memórias vívidas de coisas que nun...
3,"M, o Filho do Século",Antonio Scurati,"O romance M, o Filho do Século conta em trama ..."
4,Oblivion Song: Entre Dois Mundos,Robert Kirkman,Mestre em traçar universos distópicos permeado...
...,...,...,...
11970,Poemas completos de Alberto Caeiro,Fernando Pessoa,"Nos Poemas Completos de Alberto Caeiro (1946),..."
11971,Poesia de Álvaro de Campos,Fernando Pessoa,"Cerebral e retraído, Fernando Pessoa concebeu ..."
11972,Ben-Hur,Lewis Wallace,"O romance épico ""Ben-Hur - Uma História dos Te..."
11973,Folhas de Relva,Walt Whitman,Walt Whitman é considerado pela crítica mundia...


# Pré-processando a descrição

## Removendo números

In [60]:
df['descricao'] = df['descricao'].str.replace('\d+', '')
df['descricao']

C:\Users\TRT05\AppData\Local\Temp\ipykernel_11208\3336066140.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['descricao'] = df['descricao'].str.replace('\d+', '')


0        Quando o assunto é dinheiro, não dá para achar...
1        Elogiado por Gillian Flynn e considerado um do...
2        E se um dia memórias vívidas de coisas que nun...
3        O romance M, o Filho do Século conta em trama ...
4        Mestre em traçar universos distópicos permeado...
                               ...                        
11970    Nos Poemas Completos de Alberto Caeiro (), em ...
11971    Cerebral e retraído, Fernando Pessoa concebeu ...
11972    O romance épico "Ben-Hur - Uma História dos Te...
11973    Walt Whitman é considerado pela crítica mundia...
11974    O Sermão da Montanha é uma das mais belas mens...
Name: descricao, Length: 11975, dtype: object

## Removendo capitalização

In [61]:
df['descricao'] = df['descricao'].str.lower()
df['descricao']

0        quando o assunto é dinheiro, não dá para achar...
1        elogiado por gillian flynn e considerado um do...
2        e se um dia memórias vívidas de coisas que nun...
3        o romance m, o filho do século conta em trama ...
4        mestre em traçar universos distópicos permeado...
                               ...                        
11970    nos poemas completos de alberto caeiro (), em ...
11971    cerebral e retraído, fernando pessoa concebeu ...
11972    o romance épico "ben-hur - uma história dos te...
11973    walt whitman é considerado pela crítica mundia...
11974    o sermão da montanha é uma das mais belas mens...
Name: descricao, Length: 11975, dtype: object

## Removendo pontuação

In [62]:
df['descricao'] = df['descricao'].str.replace('[^\w\s]', '')
df['descricao']

C:\Users\TRT05\AppData\Local\Temp\ipykernel_11208\2231285731.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['descricao'] = df['descricao'].str.replace('[^\w\s]', '')


0        quando o assunto é dinheiro não dá para achar ...
1        elogiado por gillian flynn e considerado um do...
2        e se um dia memórias vívidas de coisas que nun...
3        o romance m o filho do século conta em trama e...
4        mestre em traçar universos distópicos permeado...
                               ...                        
11970    nos poemas completos de alberto caeiro  em ver...
11971    cerebral e retraído fernando pessoa concebeu o...
11972    o romance épico benhur  uma história dos tempo...
11973    walt whitman é considerado pela crítica mundia...
11974    o sermão da montanha é uma das mais belas mens...
Name: descricao, Length: 11975, dtype: object

## Removendo stopwords

In [63]:
stop = stopwords.words('portuguese')

df['descricao'] = df['descricao'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df['descricao']

0        assunto dinheiro dá achar discurso vai funcion...
1        elogiado gillian flynn considerado grandes liv...
2        dia memórias vívidas coisas nunca aconteceram ...
3        romance m filho século conta trama eletrizante...
4        mestre traçar universos distópicos permeados r...
                               ...                        
11970    poemas completos alberto caeiro versos simples...
11971    cerebral retraído fernando pessoa concebeu pro...
11972    romance épico benhur história tempos cristo pu...
11973    walt whitman considerado crítica mundial maior...
11974    sermão montanha belas mensagens literatura esp...
Name: descricao, Length: 11975, dtype: object

## Lematização

In [64]:
import spacy
nlp = spacy.load('pt_core_news_md')

In [65]:
df['descricao'] = df['descricao'].apply(lambda x: " ".join([y.lemma_ for y in nlp(x)]))
df['descricao']

0        assunto dinheiro dar achar discurso ir funcion...
1        elogiar gillian flynn considerar grande livro ...
2        dia memória vívir coisa nunca acontecer infilt...
3        romance m filho século contar trama eletrizant...
4        mestre traçar universo distópico permear refle...
                               ...                        
11970    poema completo alberto caeiro verso simples to...
11971    cerebral retrair Fernando pessoa conceber proj...
11972    romance épico benhur história tempo cristo pub...
11973    walt whitman considerar crítica mundial grande...
11974    sermão montanha belo mensagem literatura espir...
Name: descricao, Length: 11975, dtype: object

# Construindo o sistema de recomendação

## Instanciando o modelo TF-IDF

In [66]:
vectorizer = TfidfVectorizer(min_df=2, max_df=0.7)

vectorized_data = vectorizer.fit_transform(df['descricao'])

print(vectorizer.get_feature_names())

['2e', 'aa', 'aaron', 'ab', 'aba', 'abade', 'abadessa', 'abadia', 'abafado', 'abafar', 'abaixar', 'abaixo', 'abajur', 'abalado', 'abalar', 'abaler', 'abalizar', 'abalo', 'abalálo', 'abandonar', 'abandonarar', 'abandone', 'abandono', 'abandonou', 'abandonála', 'abandonálo', 'abarcar', 'abarrotar', 'abastado', 'abastar', 'abastecimento', 'abate', 'abater', 'abatir', 'abatêlo', 'abbey', 'abbeyr', 'abbi', 'abby', 'abc', 'abdicar', 'abdicação', 'abdome', 'abdução', 'abdômen', 'abe', 'abel', 'abelha', 'abelhas', 'abençoado', 'abençoadoro', 'abençoar', 'abercrombie', 'abernathy', 'aberração', 'abertamente', 'aberto', 'abertura', 'aberturar', 'aberturo', 'abhorsen', 'abigail', 'abilio', 'abismo', 'abjeto', 'able', 'abnegar', 'abnegação', 'abobalhar', 'abocanhar', 'aboler', 'abolicionista', 'abolir', 'abolição', 'abominável', 'abordagem', 'abordagemr', 'abordar', 'aborrecer', 'aborrecido', 'aborrecimento', 'abortar', 'aborto', 'aborígene', 'aborígine', 'abotoar', 'about', 'abra', 'abracar', 'ab

c:\Users\TRT05\Desktop\sistema-recomendacao\.venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Criando o dataframe do TF-IDF

In [67]:
tfidf_df = pd.DataFrame(vectorized_data.toarray(), columns = vectorizer.get_feature_names(), index = df.titulo)
tfidf_df.head()

,2e,aa,aaron,ab,aba,abade,abadessa,abadia,abafado,abafar,...,úrsula,útel,útil,𝐜𝐨𝐦,𝐝𝐞,𝐝𝐨𝐬,𝐥𝐢𝐯𝐫𝐨,𝐥𝐢𝐯𝐫𝐨𝐬,𝐦𝐚𝐢𝐬,𝐧𝐨
titulo,,,,,,,,,,,,,,,,,,,,,
Orçamento sem falhas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Minha Sombria Vanessa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Recursão,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"M, o Filho do Século",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oblivion Song: Entre Dois Mundos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Comparando as descrições com o modelo TF-IDF

In [68]:
similarity_array = linear_kernel(tfidf_df)

similarity_df = pd.DataFrame(similarity_array, index = tfidf_df.index, columns = tfidf_df.index)

print(similarity_df.head())

titulo                            Orçamento sem falhas  Minha Sombria Vanessa  \
titulo                                                                          
Orçamento sem falhas                          1.000000               0.037254   
Minha Sombria Vanessa                         0.037254               1.000000   
Recursão                                      0.015514               0.035529   
M, o Filho do Século                          0.029066               0.027313   
Oblivion Song: Entre Dois Mundos              0.019898               0.023436   

titulo                            Recursão  M, o Filho do Século  \
titulo                                                             
Orçamento sem falhas              0.015514              0.029066   
Minha Sombria Vanessa             0.035529              0.027313   
Recursão                          1.000000              0.042142   
M, o Filho do Século              0.042142              1.000000   
Oblivion Song: Entre Doi

In [75]:
similarity_series_1 = similarity_df.loc['Orçamento sem falhas']

ordered_similarities_1 = similarity_series_1.sort_values(ascending = False)

print(ordered_similarities_1[:10])

titulo
Orçamento sem falhas                                 1.000000
A Operação Pão de Queijo                             0.278410
Nunca mais fique sem dinheiro                        0.265176
Como organizar sua vida financeira                   0.234049
Gerencie o seu dinheiro como alguém foda (e-Book)    0.229749
A Psicologia do Dinheiro                             0.221570
Me Poupe!                                            0.214370
Ganhar, Gastar, Investir                             0.205843
Me Poupe!                                            0.204308
Dinheiro: os segredos de quem tem                    0.203820
Name: Orçamento sem falhas, dtype: float64


In [76]:
similarity_series_2 = similarity_df.loc['O Mundo Mágico de Harry Potter']

ordered_similarities_2 = similarity_series_2.sort_values(ascending = False)

print(ordered_similarities_2[:10])

titulo
O Mundo Mágico de Harry Potter             1.000000
Harry Potter e a Ordem da Fênix            0.277862
Harry Potter e o Enigma do Príncipe        0.263554
Harry Potter e as Relíquias da Morte       0.261668
Harry Potter e as Relíquias da Morte       0.247022
Harry Potter e o Enigma do Príncipe        0.224055
O Livro das Criaturas de Harry Potter      0.206245
Harry Potter e o Prisioneiro de Azkaban    0.203249
Harry Potter e o Prisioneiro de Azkaban    0.200783
Harry Potter e o Prisioneiro de Azkaban    0.200783
Name: O Mundo Mágico de Harry Potter, dtype: float64
